In [1]:
%pip install git+https://github.com/openai/whisper.git -q
%pip install flask-socketio
%pip install flask-ngrok
%pip install nest-asyncio
%pip install ngrok
%pip install flask-cloudflared
%pip install eventlet

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.5/53.5 kB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 46.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 226.8/226.8 kB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.7/283.7 kB 32.9 MB/s eta 0:00:00


In [2]:

import whisper
from IPython.display import Audio

small_model = whisper.load_model("small")
#medium_model = whisper.load_model("medium")
large_model = whisper.load_model("large")
#!echo "开始启动 websocket $(curl -s https://api4.ipify.org):18080"

import tornado.web
import tornado.websocket
import tornado.ioloop
import nest_asyncio
import ngrok


def transcribe(audio, model):
    # load audio and pad/trim it to fit 30 seconds
    audio = whisper.load_audio(audio)
    audio = whisper.pad_or_trim(audio)

    # make log-Mel spectrogram and move to the same device as the model
    mel = whisper.log_mel_spectrogram(audio).to(model.device)

    # detect the spoken language
    #_, probs = model.detect_language(mel)
    #print(f"Detected language: {max(probs, key=probs.get)}")

    # decode the audio
    # options = whisper.DecodingOptions()
    options = whisper.DecodingOptions(language="zh", without_timestamps=True)
    result = whisper.decode(model, mel, options)
    return result.text

100%|███████████████████████████████████████| 461M/461M [00:10<00:00, 48.0MiB/s]
100%|█████████████████████████████████████| 2.87G/2.87G [00:32<00:00, 94.3MiB/s]


In [ ]:
# TEST demo
!git clone https://github.com/petewarden/openai-whisper-webapp
easy_text = transcribe("/content/openai-whisper-webapp/mary.mp3", small_model)
print(easy_text)
hard_text = transcribe("/content/openai-whisper-webapp/daisy_HAL_9000.mp3", medium_model)
print(hard_text)

Detected language: zh
Hello,有人嗎?


In [ ]:
import json
import uuid
import os

from flask import Flask, request
from flask_socketio import SocketIO, emit
from flask_cloudflared import run_with_cloudflared
from flask_ngrok import run_with_ngrok

name_space = '/websocket'  # 名称空间，写对才能通信
app = Flask(__name__, template_folder="template")
app.secret_key = 'secret!'
socketio = SocketIO(app, cors_allowed_origins='*')  # 允许跨域
client_query = []  # 这里也可以用自带的join_room之类的方法
client_handshakes = {
}

@app.route("/")
def index():
    # 返回一个字符串作为响应
    return "This is the index page."

@socketio.on('connect', namespace=name_space)  # 有客户端连接会触发该函数
def on_connect():
    global client_handshakes
    # 建立连接 sid:连接对象ID
    client_id = request.sid
    client_query.append(client_id)
    client_handshakes[client_id] = {}
    # emit(event_name, broadcasted_data, broadcast=False, namespace=name_space, room=client_id)  #指定一个客户端发送消息
    # emit(event_name, broadcasted_data, broadcast=True, namespace=name_space)  #对name_space下的所有客户端发送消息
    print(u'new connection,id=[%s] connected, now have [%s] connections' % (client_id, len(client_query)))


@socketio.on('disconnect', namespace=name_space)  # 有客户端断开WebSocket会触发该函数
def on_disconnect():
    global client_handshakes
    # 连接对象关闭 删除对象ID
    client_query.remove(request.sid)
    if request.sid in client_handshakes:
      del(client_handshakes[request.sid])
    print(u'connection id=[%s] exited, now have [%s] connections' % (request.sid, len(client_query)))


# on('消息订阅对象', '命名空间区分')
@socketio.on('message', namespace=name_space)
def on_message(message):
    global client_handshakes
    """ 服务端接收消息 """
    print(u'从id=%s客户端中收到消息，内容如下:' % request.sid)
    # print(message)
    # 指定一个客户端发送消息
    # emit('my_response_message', "我收到了你的信息", broadcast=False, namespace=name_space, room= request.sid)
    # 对name_space下的所有客户端发送消息
    # emit('my_response_message', u"我收到了你的信息", broadcast=True, namespace=name_space)
    if isinstance(message, str):
        print('Is str.')
        obj = json.loads(message)
        print(message, obj)
        client_handshakes[request.sid] = obj
        emit('raw_message', '{"ok":true}')

    if isinstance(message, bytes):
        if 'type' not in client_handshakes[request.sid] or client_handshakes[request.sid]['type'] == "":
            emit('error', '{"ok":false, "message": "请先发送类型"}')
            return
        t = "small"
        if 'model' in client_handshakes[request.sid]:
            t = client_handshakes[request.sid]['model']
        print('Is bytes.')
        path = u"/content/cache/%s.%s" % (uuid.uuid1(), client_handshakes[request.sid]['type'])
        with open(path, 'wb') as f:
            f.write(message)
        model = small_model
        #model = model if t == "medium" else medium_model
        model = model if t == "large" else large_model
        hard_text = transcribe(path, model)
        print(hard_text)
        emit('raw_message', u'{"ok":true, "message":"%s"}' % hard_text)
        os.remove(path)



if __name__ == '__main__':
    if not os.path.isdir('/content/cache'):
        os.makedirs('/content/cache')
    run_with_cloudflared(socketio)
    #run_with_ngrok(socketio)
    socketio.run(app)


# const sc = document.createElement("script");
# sc.src = "https://cdn.bootcdn.net/ajax/libs/socket.io/4.6.1/socket.io.min.js";
# document.body.appendChild(sc);

# var ws = io.connect("wss://"+document.domain + ":" + location.port + "/websocket")
# ws.on('my_response_message', function (params) {
#     console.log(params)
# })
# ws.on('raw_message', function (params) {
#     console.log(params)
# })
#
# ws.send("234")

# 发送文件
# var reader = new FileReader();
# reader.onload = function (e) {
#   var blob = new Blob([new Uint8Array(e.target.result)], { type: 'application/octet-stream' });
#   ws.send(blob);
# };
# reader.readAsArrayBuffer(document.querySelector("#file").files[0]);

 * Downloading cloudflared for Linux x86_64...
 * Running on https://fotos-cumulative-commodities-dover.trycloudflare.com
 * Traffic stats available on http://127.0.0.1:8794/metrics
new connection,id=[UIyuWrwV4SjTzSx3AAAB] connected, now have [1] connections
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is str.
{"type":"wav","model":"large"} {'type': 'wav', 'model': 'large'}
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
太長包了
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
Hello
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
嗯
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
你再回放我啊
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
後續內容如下
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
字幕by索兰娅
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
現狀醒來發現自己被反在椅子上你準備開始辱罵我
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
字幕by索兰娅
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
我們還有字幕 白索 狼鴨
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is bytes.
這什麼鬼啊 用什麼數據訊息的東西
从id=UIyuWrwV4SjTzSx3AAAB客户端中收到消息，内容如下:
Is b